In [3]:
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
import imageio
import imutils

ModuleNotFoundError: No module named 'imutils'

In [4]:
# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels only
def color_thresh(img, rgb_thresh=(160, 160, 160)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                & (img[:,:,1] > rgb_thresh[1]) \
                & (img[:,:,2] > rgb_thresh[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

In [5]:
# Define a function to convert from image coords to rover coords
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[1]/2 ).astype(np.float)
    return x_pixel, y_pixel

In [6]:

# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

In [7]:
# Define a function to map rover space pixels to world space
def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated


In [8]:
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

In [9]:
def color_thresh_range(img, rgb_lower, rgb_higher):
    """
    takes an image input, and returns an image composed of colors within range
    :param img:
    :param rgb_lower: lower range
    :param rgb_higher: higher range
    :return:
    """
    color_select = np.zeros_like(img[:, :, 0])
    above_thresh = (img[:, :, 0] > rgb_lower[0]) \
                   & (img[:, :, 1] > rgb_lower[1]) \
                   & (img[:, :, 2] > rgb_lower[2])
    below_thresh = (img[:, :, 0] < rgb_higher[0]) \
                   & (img[:, :, 1] < rgb_higher[1]) \
                   & (img[:, :, 2] < rgb_higher[2])

    color_select[above_thresh & below_thresh] = 255
    return color_select

In [10]:
# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

In [11]:
def hsv_color_thresh_range(img, hsv_lower, hsv_higher):
    # convert rgb image to hsv
    converted = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Convert BGR to HSV

    color_select = color_thresh_range(converted, hsv_lower, hsv_higher)
    return color_select

In [12]:
# Define a function to perform a perspective transform
def perspect_transform(img, src, dst):
           
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    return warped


In [13]:
# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change the path below to your data directory
# If you are in a locale (e.g., Europe) that uses ',' as the decimal separator
# change the '.' to ','
df = pd.read_csv('../test_dataset/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = 0 # This will be a running index
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()


ModuleNotFoundError: No module named 'pandas'

In [14]:
def process_image(img):

    image = img    
    # TODO: 
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    # threshold the image before warping, to avoid artifacts in the resulting image
    nav_lower_range = (180, 180, 180)
    nav_upper_range = (256, 256, 256)
    navigable_terrain = color_thresh_range(image, nav_lower_range, nav_upper_range)

    obstacle_lower_range = (0, 0, 0)
    obstacle_upper_range = (179, 179, 179)
    obstacle_terrain = color_thresh_range(image, obstacle_lower_range, obstacle_upper_range)

    rock_sample_thresh = color_thresh_range(image, (110, 110, 0), (255, 255, 50))
    
    
    # 1) Define source and destination points for perspective transform
    # source points
    rows, cols, ch = image.shape
    # lower 13, 140 and 303,140
    # higher 118, 95 and 200, 95
    dst_size = 15
    # Set a bottom offset to account for the fact that the bottom of the image
    # is not the position of the rover but a bit in front of it
    # this is just a rough guess, feel free to change it!
    bottom_offset = 5
    source = np.float32([[14, 140],
    [300, 140],
    [200, 95],
    [120, 95]])
    destination = np.float32([[image.shape[1] / 2 - dst_size, image.shape[0] - bottom_offset],
                              [image.shape[1] / 2 + dst_size, image.shape[0] - bottom_offset],
                              [image.shape[1] / 2 + dst_size, image.shape[0] - 2 * dst_size - bottom_offset],
                              [image.shape[1] / 2 - dst_size, image.shape[0] - 2 * dst_size - bottom_offset],
                              ])    
    # 2) Apply perspective transform    
    warped_navigable_terrain = perspect_transform(navigable_terrain, source, destination)
    warped_obstacle_terrain = perspect_transform(obstacle_terrain, source, destination)
    warped_rock_sample_thresh = perspect_transform(rock_sample_thresh, source, destination)
    warped_navigable_terrain_rgb = cv2.cvtColor(warped_navigable_terrain, cv2.COLOR_GRAY2RGB)
    warped_obstacle_terrain_rgb = cv2.cvtColor(warped_obstacle_terrain, cv2.COLOR_GRAY2RGB)
    warped_navigable_terrain_rgb_rotated=imutils.rotate(warped_navigable_terrain_rgb, -90)
    # 4) Convert thresholded image pixel values to rover-centric coords
    obstacle_xpix, obstacle_ypix = rover_coords(warped_obstacle_terrain)
    navigable_xpix, navigable_ypix = rover_coords(warped_navigable_terrain)
    rock_sample_xpix, rock_sample_ypix = rover_coords(warped_rock_sample_thresh)        
    
    # 5) Convert rover-centric pixel values to world coords
    scale = 2*dst_size
    obstacle_x_world, obstacle_y_world = pix_to_world(obstacle_xpix, obstacle_ypix, data.xpos[data.count],
                                                      data.ypos[data.count], data.yaw[data.count],
                                                      data.worldmap.shape[0], scale)

    rock_sample_x_world, rock_sample_y_world = pix_to_world(rock_sample_xpix, rock_sample_ypix, data.xpos[data.count],
                                                            data.ypos[data.count], data.yaw[data.count],
                                                            data.worldmap.shape[0], scale)

    x_world, y_world = pix_to_world(navigable_xpix, navigable_ypix, data.xpos[data.count],
                                    data.ypos[data.count], data.yaw[data.count],
                                    data.worldmap.shape[0], scale)
    y=img.shape[0]
    x=img.shape[1]
    data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 15
    data.worldmap[rock_sample_y_world, rock_sample_x_world, :] += 255
    data.worldmap[y_world, x_world, 2] += 255        
                
    # 7) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*3, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
        
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, first a warped image
    warped = perspect_transform(img, source, destination)
        # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], x:2*x] = warped
    warped_obstacle_terrain_rgb[:,:,1:3]-=255
    warped_navigable_terrain_rgb_rotated[:,:,1]-=255
    warped_navigable_terrain_rgb_rotated[:,:,0:1]-=255

    output_image[0:img.shape[0], 2*x:3*x] = warped_navigable_terrain_rgb
    output_image[img.shape[0]:2*y, 2*x:3*x]=warped_obstacle_terrain_rgb
    output_image[img.shape[0]:2*y, x:2*x]=warped_navigable_terrain_rgb_rotated
        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:map_add.shape[1]]= np.flipud(map_add)


        # Then putting some text over the image
    cv2.putText(output_image,"Output flow Debugging mode video", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)
    cv2.putText(output_image,"Map", (20, 315), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)
    cv2.putText(output_image,"Rover perciption", (340, 315), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)
    cv2.putText(output_image,"Bird eye view", (340, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)
    cv2.putText(output_image,"navigable terrain", (640, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)
    cv2.putText(output_image,"obstacle terrain", (620, 315), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 0, 255), 1)    
    if data.count < len(data.images) - 1:
        data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

ModuleNotFoundError: No module named 'moviepy'

In [67]:
from IPython.display import HTML
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))